In [21]:
#importing matplotlib.pyplot to generate the frames of the gif
import matplotlib.pyplot as plt
import numpy as np
#importing scipy.integrate and scipy.optimize to calculate the percent overlap
import scipy.integrate as integrate
import scipy.optimize as optimize
#HSV to RGB function used to make the colors pretty
from matplotlib.colors import hsv_to_rgb

In [2]:
#takes in a vector (x_in, y_in) and rotates it by theta radians
def rotate(x_in, y_in, theta):
    x_out = x_in*np.cos(theta)-y_in*np.sin(theta)
    y_out = x_in*np.sin(theta)+y_in*np.cos(theta)
    return x_out, y_out

In [3]:
#takes in an inital point (x_i, y_i) and a final point (x_f, y_f)
#then makes a straight line connectin the points
# t = 0 returns the intial point
# t = 1 returns th final point
def translate(x_i, y_i, x_f, y_f, t):
    dx = x_f - x_i
    dy = y_f - y_i
    return dx * t + x_i, dy * t + y_i

In [4]:
#this function is designed to calculate the values
#needed for the pyplot.arrow() function
#it makes the arrows parallel to a slope
def arrowMath(x_base, x_head, slope, height): 
    y_base = slope*x_base + height
    y_head = slope*x_head + height
    
    return x_base, y_base, x_head-x_base, y_head-y_base

In [5]:
#draws location of the boxes for the crab chambers (dotted)
def drawCrabChamber(x, y):
    ax.plot([x[0], x[1]],[y[0], y[1]],'black', ls = ':')
    ax.plot([x[1], x[2]],[y[1], y[2]],'black', ls = ':')
    ax.plot([x[2], x[3]],[y[2], y[3]],'black', ls = ':')
    ax.plot([x[3], x[0]],[y[3], y[0]],'black', ls = ':')

In [6]:
#ellipse (long)
def ellipse(t):
    if( t < -2 ):
        return 0
    elif( t > -2 and t < 2 ):
        return 0.6*np.sqrt( 1 - (t/2)**2 )
    elif( t > 2 ):
        return 0
    
#area of the ellipse
area_of_ellipse = 2*integrate.quad(ellipse, -2, 2)[0]

#area of the overlap
def area_of_overlap(t):
    if( t <= -2 ):
        return 0
    elif( t > -2 and t <= 0):
        return 4*integrate.quad(lambda x : ellipse(x-t), 0, t+2)[0]
    elif( t > 0 and t <= 2):
        return 4*integrate.quad(lambda x : ellipse(x-t), t-2, 0)[0]
    elif(t > 2):
        return 0

#percent overlap
def percent_overlap(t):
    return area_of_overlap(t) / ( 2*area_of_ellipse - area_of_overlap(t) )

tau = np.linspace(-3, 3, 100)
overlap = []
for t in tau:
    overlap.append(percent_overlap(t))

overlap = np.array(overlap)

In [7]:
#ellipse (tall)
def ellipse2(t):
    if( t <= -0.6 ):
        return 0
    elif( t > -0.6 and t <= 0.6 ):
        return 2*np.sqrt( 1 - (t/0.6)**2 )
    elif( t > 0.6 ):
        return 0

#area of the overlap
def area_of_overlap2(t):
    if( t <= -2.6 ):
        return 0
    elif( t > -2.6 and t <= -1.4 ):
        intersection = optimize.root(lambda x : ellipse(x - t) - ellipse2(x), -0.6).x
        integral_1 = 2*integrate.quad(ellipse2, -0.6, intersection)[0]
        integral_2 = 2*integrate.quad(lambda x : ellipse(x-t), intersection, t+2)[0]
        return integral_1 + integral_2
    elif( t > -1.4 and t<= 0 ):
        intersection_1 = optimize.root(lambda x : ellipse(x - t) - ellipse2(x), -0.6).x
        intersection_2 = optimize.root(lambda x : ellipse(x - t) - ellipse2(x), 0.6).x
        integral_1 = 2*integrate.quad(ellipse2, -0.6, intersection_1)[0]
        integral_2 = 2*integrate.quad(lambda x : ellipse(x-t), intersection_1, intersection_2)[0]
        integral_3 = 2*integrate.quad(ellipse2, intersection_2, 0.6)[0]
        return integral_1 + integral_2 + integral_3
    elif( t > 0 ):
        return area_of_overlap2(-t)

#percent overlap
def percent_overlap2(t):
    return area_of_overlap2(t) / ( 2*area_of_ellipse - area_of_overlap2(t) )
    
tau = np.linspace(-3, 3, 100)
overlap2 = []
for t in tau:
    overlap2.append(percent_overlap2(t))

overlap2 = np.array(overlap2)

In [8]:
#the crab chamber pulsating arrows locations and height
# isFirst = True draws the arrows for the first crab chamber
# isFirst = False draws the arrows for the second crab chamber
def drawCrabbingArrows(percent, isFirst):
    
    flip = 1
    if(isFirst):
        flip = 1
    else:
        flip = -1
    
    def drawCrabbingArrow(y_base, dx):
        alpha = np.arctan2(2,1)
        
        x_base = 0
        dy = 0
        x_base, y_base = rotate(x_base, y_base, alpha)
        dx, dy = rotate(dx, dy, alpha)
        delta_x, delta_y = translate(-20, 10, 20, -10, percent)
        x_base += delta_x
        y_base += delta_y
        ax.arrow(x_base, y_base, dx, dy, color='black', width = 0.08)
        ax.arrow(-x_base, y_base, -dx, dy, color='black', width = 0.08)
    
    
    drawCrabbingArrow(flip*1.5, flip*0.72)
    drawCrabbingArrow(flip*1.0, flip*0.48)
    drawCrabbingArrow(flip*0.5, flip*0.24)
    drawCrabbingArrow(-flip*0.5, -flip*0.24)
    drawCrabbingArrow(-flip*1.0, -flip*0.48)
    drawCrabbingArrow(-flip*1.5, -flip*0.72)

In [27]:
## MOO MOO MOO MOO CODE STARTS HERE MOO MOO MOO MOO
frames = np.linspace(0, 1, 300)
phi = np.linspace(0, 2*np.pi, 100)

x = .6*np.cos(phi)
y = 2*np.sin(phi)


frequency = 50

count = 0
repeat = [-6, -5, -4, -3, -2, -1, 0]
for t in frames:
    if(count % frequency == 0):
        repeat.append( repeat[-1] + 1 )
    
    #setting up the graph
    fig = plt.figure(figsize = (16,9))
    ax = fig.add_subplot(111)
    ax.set_xlim(-20,20)
    ax.set_ylim(-10,10) 
    plt.axis('off')
    
    #phi_cross
    phicross = np.linspace(-1.18*np.pi/8, 1.18*np.pi/8, 100)
    ax.plot( 10*np.cos(phicross), 10*np.sin(phicross), 'black', ls = '--', alpha = 0.5)
    text_phiCross = ax.text(11, 0, '$\phi_{cross}= 25$ mrad', color='black')
    text_phiCross.set_fontsize(15.2)

    #This is for percent overlap text
    current_overlap = 0.0
    
    for r in repeat:
        percent = t - r*50/300
        
        #drawing the ellipse
        x1 = x
        y1 = y

        alpha = np.arctan2(2,1)
        beta = -np.pi/2 + 2*alpha
        if( percent <= 0.20 ):
            #diagonal rotation
            x1, y1 = rotate(x1, y1, alpha)
            
        elif( percent > 0.20 and percent <= 0.30 ):
            #palpha to beta rotation
            slope = (beta - alpha)/(0.30 - 0.20)
            x1, y1 = rotate(x1, y1, slope*(percent - 0.20) + alpha)
        elif( percent > 0.30 and percent <= 0.40 ):
            #beta to alpha rotation
            slope = (alpha - beta)/(0.40 - 0.30)
            x1, y1 = rotate(x1, y1, slope*(percent - 0.30) + beta)
        elif( percent > 0.40 and percent <= 0.50 ):
            #alpha to pi/2 rotation
            slope = (np.pi/2 - alpha)/(0.50 - 0.40)
            x1, y1 = rotate(x1, y1, slope*(percent - 0.40) + alpha)
        
        elif( percent > 0.50 and percent <= 0.60 ):
            #pi/2 to alpha rotation
            slope = (alpha - np.pi/2)/(0.60 - 0.50)
            x1, y1 = rotate(x1, y1, slope*(percent - 0.50) + np.pi/2)
        elif( percent > 0.60 and percent <= 0.70 ):
            #alpha to beta rotation
            slope = (beta - alpha)/(0.70 - 0.60)
            x1, y1 = rotate(x1, y1, slope*(percent - 0.60) + alpha)
        elif( percent > 0.70 and percent <= 0.80 ):
            #beta to alpha rotation
            slope = (alpha - beta)/(0.80 - 0.70)
            x1, y1 = rotate(x1, y1, slope*(percent - 0.70) + beta)
            
        elif( percent > 0.80 ):
            #diagonal rotation
            x1, y1 = rotate(x1, y1, alpha)
        
        if( percent > 0.185 and percent < 0.215):
            #draw crabbing arrows
            drawCrabbingArrows(0.20, True)
        
        if( percent > 0.785 and percent < 0.815):
            drawCrabbingArrows(0.80, False)
        
        if( percent > 0.425 and percent < 0.575 ):
            #Percent overlap
            #there are 100 overlap values
            #the 0.40-0.60 range has to be mapped to the 0-99 range
            # "percent - 0.40" gives the fixed range, 0.00-0.20
            # 100/0.20 gives the ratio that changes
            current_overlap = overlap[ int((percent - 0.425) * 100/(0.575-0.425)) ]
        
        #translation
        x_i = -20
        y_i = 10
        x_f = 20
        y_f = -10
        delta_x, delta_y = translate(x_i, y_i, x_f, y_f, percent)
        x1 += delta_x
        y1 += delta_y
        
        #math for color of fill
        # blue:   #0b14c1
        # red:    #c10b11
        # purple: #880bc1
        b_hue = 237.0
        r_hue = 358.0
        p_hue = 281.0
        #cosine interpolation from hue1 --> hue2 --> hue1
        def cosineColor(hue1, hue2, percent):
            return (hue1 - hue2)/2.0 * np.cos(2*np.pi*percent) + (hue1 + hue2)/2.0
        
        #get current hues
        current_blue_hue = cosineColor(b_hue, p_hue, percent)
        current_red_hue = cosineColor(r_hue, p_hue, percent)
        
        #convert HSV to RGB
        current_blue = hsv_to_rgb((current_blue_hue/360, 0.94, 0.76))
        current_red = hsv_to_rgb((current_red_hue/360, 0.94, 0.76))
        
        #plotting the curve and filling
        ax.fill(x1,y1, facecolor = current_blue, alpha = .85, zorder = 1)
        ax.fill(-x1,y1, facecolor = current_red, alpha = .85, zorder = 1)
        
    #path the beams follow
    ax.plot([-30, 30],[15, -15],'#880bc1', alpha = 0.7, zorder = -1)
    ax.plot([30, -30],[15, -15],'#880bc1', alpha = 0.7, zorder = -1)
    
    #Percent Overlap text
    overlap_percent = 100*current_overlap
    text_percentOverlap = ax.text(0, -4.5,
                                  '%%-Overlap: %0.2f' % overlap_percent,
                                  color='black', ha = 'center')
    text_percentOverlap.set_fontsize(16)
    
    #Percent Overlap gauge
    # (-a,b)-----(a,b)
    #    |         |
    # (-a,c)-----(a,c)
    a = 6.25
    b = -5
    c = -6
    gauge_x = [a, -a, -a, a, a]
    gauge_y = [b, b, c, c, b]
    
    ax.plot(gauge_x, gauge_y, 'black')
    gauge_x_percent = gauge_x
    gauge_x_percent[0] = gauge_x_percent[-1] = gauge_x_percent[-2] = 2*a*current_overlap - a
    ax.fill(gauge_x_percent, gauge_y, facecolor='#880bc1')
    
    #Arrows
    alpha = np.degrees(np.arctan2(1,2))
    
    x_base, y_base, dx, dy = arrowMath(-10, -4.25, -1/2, 1.75)
    ax.arrow(x_base, y_base, dx, dy, color = 'black', shape = 'right', width = 0.25)
    x_base, y_base, dx, dy = arrowMath(-10, -4.25, -1/2, 2.2)
    text_ionBeam = ax.text(x_base + dx/2, y_base + dy/2, 'Ion beam',
                           color='#0b14c1', rotation = -alpha, wrap=True, ha = 'center')
    text_ionBeam.set_fontsize(16)
    
    x_base, y_base, dx, dy = arrowMath(10, 4.25, 1/2, 1.75)
    ax.arrow(x_base, y_base, dx, dy, color = 'black', shape = 'left', width = 0.25)
    x_base, y_base, dx, dy = arrowMath(10, 4.25, 1/2, 2.2)
    text_eBeam = ax.text(x_base + dx/2, y_base + dy/2, 'e-beam',
                         color='#c10b11', rotation = alpha, wrap=True, ha = 'center')
    text_eBeam.set_fontsize(16)
    
    #BoxeSettings
    x_i = -20
    y_i = 10
    x_f = 20
    y_f = -10
    alpha = np.arctan2(2,1)
    beta = np.degrees(np.arctan2(1,2))
    x_box = np.array([1.25, -1.25, -1.25, 1.25])
    y_box = np.array([2.0, 2.0, -2.0, -2.0])
    x_box, y_box = rotate(x_box, y_box, alpha)
    delta_x, delta_y = translate(x_i, y_i, x_f, y_f, 0.20)
    x_box += delta_x
    y_box += delta_y
    #top-right
    drawCrabChamber(x_box, y_box)
    #bottom-right
    drawCrabChamber(x_box, -y_box)
    text_bottomright = ax.text(-x_box[1]-6.25, -y_box[0]+1 , 'Bunch De-crabbing',
                           color='black', rotation = -beta, wrap=True)
    text_bottomright.set_fontsize(16)
    #top-left
    drawCrabChamber(-x_box, y_box)
    text_topleft = ax.text(x_box[1]-1.5, y_box[0]-3 , 'Bunch Crabbing',
                           color='black', rotation = -beta, wrap=True)
    text_topleft.set_fontsize(16)
    #bottom-left
    drawCrabChamber(-x_box, -y_box)
    
    #signature
    text_signature = ax.text(-12.5,-9.75,'By A.J Parker At Jefferson Lab', color = 'black', ha = 'center')
    text_signature.set_fontsize(14)
    
    #title
    text_title = ax.text(0, 8.25, 'Crab Crossing', color='black', ha = 'center')
    text_title.set_fontsize(26)
    
    #saving the figure
    fig.savefig('Warning2DCrabs/CC2D_%03d.jpeg' % count, 
                bbox_inches = 'tight', pad_inches = 0)
    plt.close(fig)
    count += 1

print('Finished!')

Finished!
